# Data Processing

In [ ]:
# Adding Path of each Image
img_path = []
for row in total_data.iterrows():
    path = "/content/adience-benchmark-gender-and-age-classification/AdienceBenchmarkGenderAndAgeClassification/faces/"+row[1].user_id+"/coarse_tilt_aligned_face."+str(row[1].face_id)+"."+row[1].original_image
    img_path.append(path)
df['img_path'] = img_path

In [ ]:
df.head()

,age,gender,x,y,dx,dy,img_path
0,"(25, 32)",f,0,414,1086,1383,/content/adience-benchmark-gender-and-age-clas...
1,"(25, 32)",m,301,105,640,641,/content/adience-benchmark-gender-and-age-clas...
2,"(25, 32)",f,2395,876,771,771,/content/adience-benchmark-gender-and-age-clas...
3,"(25, 32)",m,752,1255,484,485,/content/adience-benchmark-gender-and-age-clas...
4,"(25, 32)",m,175,80,769,768,/content/adience-benchmark-gender-and-age-clas...


In [ ]:
# How many unique Age are there
df.age.unique()

array(['(25, 32)', '(38, 43)', '(4, 6)', '(60, 100)', '(15, 20)',
       '(48, 53)', '(8, 12)', '(0, 2)', 'None', '(38, 48)', '35', '3',
       '55', '58', '22', '13', '45', '36', '23', '(38, 42)', '(8, 23)',
       '(27, 32)', '57', '56', '2', '29', '34', '42', '46', '32'],
      dtype=object)

In [ ]:
# Mapping Ages to bring All the Ages into there respective Range
age_mapping = [('(0, 2)', '0-2'), ('2', '0-2'), ('3', '0-2'), ('(4, 6)', '4-6'), ('(8, 12)', '8-13'), ('13', '8-13'), ('22', '15-20'), ('(8, 23)','15-20'), ('23', '25-32'), ('(15, 20)', '15-20'), ('(25, 32)', '25-32'), ('(27, 32)', '25-32'), ('32', '25-32'), ('34', '25-32'), ('29', '25-32'), ('(38, 42)', '38-43'), ('35', '38-43'), ('36', '38-43'), ('42', '48-53'), ('45', '38-43'), ('(38, 43)', '38-43'), ('(38, 42)', '38-43'), ('(38, 48)', '48-53'), ('46', '48-53'), ('(48, 53)', '48-53'), ('55', '48-53'), ('56', '48-53'), ('(60, 100)', '60+'), ('57', '60+'), ('58', '60+')]
age_mapping_dict = {each[0]: each[1] for each in age_mapping}
drop_labels = []
for idx, each in enumerate(df.age):
    if each == 'None':
        drop_labels.append(idx)
    else:
        df.age.loc[idx] = age_mapping_dict[each]
df = df.drop(labels=drop_labels, axis=0) #droped None values
df.age.value_counts(dropna=False)

<ipython-input-22-cd61abbbee4f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.age.loc[idx] = age_mapping_dict[each]


25-32    5296
38-43    2776
0-2      2509
8-13     2292
4-6      2140
15-20    1792
48-53     916
60+       901
Name: age, dtype: int64

In [ ]:
# Dropping Null Values
df = df.dropna()
unbiased_data = df[df.gender != 'u'].copy()
unbiased_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17452 entries, 0 to 19345
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   age       17452 non-null  object
 1   gender    17452 non-null  object
 2   x         17452 non-null  int64 
 3   y         17452 non-null  int64 
 4   dx        17452 non-null  int64 
 5   dy        17452 non-null  int64 
 6   img_path  17452 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.1+ MB


In [ ]:
gender_to_label_map = {
    'f' : 0,
    'm' : 1
}

age_to_label_map = {
    '0-2'  :0,
    '4-6'  :1,
    '8-13' :2,
    '15-20':3,
    '25-32':4,
    '38-43':5,
    '48-53':6,
    '60+'  :7
}

label_to_age_map = {v: k for k, v in age_to_label_map.items()}

# label_to_age_map = {value: key for key, value in age_to_label_map.items()}
# label_to_gender_map = {value: key for key, value in gender_to_label_map.items()}

unbiased_data['age'] = unbiased_data['age'].apply(lambda age: age_to_label_map[age])
unbiased_data['gender'] = unbiased_data['gender'].apply(lambda g: gender_to_label_map[g])
unbiased_data.head()

,age,gender,x,y,dx,dy,img_path
0,4,0,0,414,1086,1383,/content/adience-benchmark-gender-and-age-clas...
1,4,1,301,105,640,641,/content/adience-benchmark-gender-and-age-clas...
2,4,0,2395,876,771,771,/content/adience-benchmark-gender-and-age-clas...
3,4,1,752,1255,484,485,/content/adience-benchmark-gender-and-age-clas...
4,4,1,175,80,769,768,/content/adience-benchmark-gender-and-age-clas...


In [ ]:
# Unique Ages
unbiased_data.age.unique()

array([4, 5, 1, 7, 3, 6, 2, 0])

In [ ]:
# Splitting the Data into Train and test set
X = unbiased_data[['img_path']]
y = unbiased_data[['gender']]
yy=keras.utils.to_categorical(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('Train data shape {}'.format(X_train.shape))
print('Test data shape {}'.format(X_test.shape))

Train data shape (12216, 1)
Test data shape (5236, 1)


In [ ]:
# Remove the Index from X_train
X_train=X_train.reset_index()
X_train=X_train.drop('index',axis=1)
X_train

,img_path
0,/content/adience-benchmark-gender-and-age-clas...
1,/content/adience-benchmark-gender-and-age-clas...
2,/content/adience-benchmark-gender-and-age-clas...
3,/content/adience-benchmark-gender-and-age-clas...
4,/content/adience-benchmark-gender-and-age-clas...
...,...
12211,/content/adience-benchmark-gender-and-age-clas...
12212,/content/adience-benchmark-gender-and-age-clas...
12213,/content/adience-benchmark-gender-and-age-clas...
12214,/content/adience-benchmark-gender-and-age-clas...


In [ ]:
# Remove the index from y_train
y_train=y_train.reset_index()
y_train=y_train.drop('index',axis=1)
y_train

,gender
0,1
1,0
2,0
3,1
4,0
...,...
12211,1
12212,0
12213,0
12214,0


In [ ]:
# Creating The Data Frame for train images and 
train=pd.DataFrame({'Image_path':X_train.iloc[:,0],'Gender':y_train.iloc[:,0]},index=X_train.index)
train

,Image_path,Gender
0,/content/adience-benchmark-gender-and-age-clas...,1
1,/content/adience-benchmark-gender-and-age-clas...,0
2,/content/adience-benchmark-gender-and-age-clas...,0
3,/content/adience-benchmark-gender-and-age-clas...,1
4,/content/adience-benchmark-gender-and-age-clas...,0
...,...,...
12211,/content/adience-benchmark-gender-and-age-clas...,1
12212,/content/adience-benchmark-gender-and-age-clas...,0
12213,/content/adience-benchmark-gender-and-age-clas...,0
12214,/content/adience-benchmark-gender-and-age-clas...,0


In [ ]:
# Doing the Same for test Data 
X_test=X_test.reset_index()
X_test=X_test.drop('index',axis=1)
y_test=y_test.reset_index()
y_test=y_test.drop('index',axis=1)
X_test,y_test

(                                               img_path
 0     /content/adience-benchmark-gender-and-age-clas...
 1     /content/adience-benchmark-gender-and-age-clas...
 2     /content/adience-benchmark-gender-and-age-clas...
 3     /content/adience-benchmark-gender-and-age-clas...
 4     /content/adience-benchmark-gender-and-age-clas...
 ...                                                 ...
 5231  /content/adience-benchmark-gender-and-age-clas...
 5232  /content/adience-benchmark-gender-and-age-clas...
 5233  /content/adience-benchmark-gender-and-age-clas...
 5234  /content/adience-benchmark-gender-and-age-clas...
 5235  /content/adience-benchmark-gender-and-age-clas...
 
 [5236 rows x 1 columns],       gender
 0          1
 1          1
 2          1
 3          1
 4          1
 ...      ...
 5231       0
 5232       0
 5233       0
 5234       1
 5235       1
 
 [5236 rows x 1 columns])

In [ ]:
# Create the Data Frame for test Data or Validation Data 
test=pd.DataFrame({'Image_path':X_test.iloc[:,0],'Label':y_test.iloc[:,0]},index=X_test.index)
test

,Image_path,Label
0,/content/adience-benchmark-gender-and-age-clas...,1
1,/content/adience-benchmark-gender-and-age-clas...,1
2,/content/adience-benchmark-gender-and-age-clas...,1
3,/content/adience-benchmark-gender-and-age-clas...,1
4,/content/adience-benchmark-gender-and-age-clas...,1
...,...,...
5231,/content/adience-benchmark-gender-and-age-clas...,0
5232,/content/adience-benchmark-gender-and-age-clas...,0
5233,/content/adience-benchmark-gender-and-age-clas...,0
5234,/content/adience-benchmark-gender-and-age-clas...,1


# Data Split

In [ ]:
from keras.utils import to_categorical

In [ ]:
n_classes = 8
column_labels_name = [label_to_age_map[x] for x in range(n_classes)]
column_labels_name

['0-2', '4-6', '8-13', '15-20', '25-32', '38-43', '48-53', '60+']

In [ ]:
x = unbiased_data[['img_path']]
y = unbiased_data[['age']]
#y=keras.utils.to_categorical(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print('Train data shape {}'.format(X_train.shape))
print('test data shape {}'.format(X_test.shape))

Train data shape (12216, 1)
test data shape (5236, 1)


In [ ]:
one_hot = pd.DataFrame(to_categorical(y_train["age"], n_classes))
  
y_train=y_train.reset_index()
y_train=y_train.drop('index',axis=1)

one_hot.columns = column_labels_name # change labels name
y_train = y_train.drop("age", axis = 1) # drop old not hot-encoded columns 
y_train = pd.concat([y_train, one_hot], axis=1) # add new hot-encoded columns 


y_train

,0-2,4-6,8-13,15-20,25-32,38-43,48-53,60+
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
12211,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12212,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12213,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12214,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
y_test=y_test.reset_index()
y_test=y_test.drop('index',axis=1)

one_hot = pd.DataFrame(to_categorical(y_test["age"], n_classes))

one_hot.columns = column_labels_name # change labels name
y_test = y_test.drop("age", axis = 1) # drop old not hot-encoded columns 
y_test = pd.concat([y_test, one_hot], axis=1) # add new hot-encoded columns 


y_test

,0-2,4-6,8-13,15-20,25-32,38-43,48-53,60+
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
5231,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5232,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5233,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5234,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
test.columns=['Image_path','Gender_label']
train.columns=['Image_path','Gender_label']
# test.columns=['Image_path','Gender_label']

In [ ]:
train.head()

,Image_path,Gender_label
0,/content/adience-benchmark-gender-and-age-clas...,1
1,/content/adience-benchmark-gender-and-age-clas...,0
2,/content/adience-benchmark-gender-and-age-clas...,0
3,/content/adience-benchmark-gender-and-age-clas...,1
4,/content/adience-benchmark-gender-and-age-clas...,0


In [ ]:
# train_t=train.copy()
# test_t=test.copy()

In [ ]:
train[column_labels_name]=y_train
test[column_labels_name]=y_test

In [ ]:
# train_t.head()

In [ ]:
train.head()

,Image_path,Gender_label,0-2,4-6,8-13,15-20,25-32,38-43,48-53,60+
0,/content/adience-benchmark-gender-and-age-clas...,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,/content/adience-benchmark-gender-and-age-clas...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,/content/adience-benchmark-gender-and-age-clas...,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/content/adience-benchmark-gender-and-age-clas...,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,/content/adience-benchmark-gender-and-age-clas...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
# os.chdir('/kaggle/working')

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')